In [6]:
import fxcmpy
import pandas as pd
import datetime as dt
import json
import ta

In [2]:
con = fxcmpy.fxcmpy(access_token="d808da394895605e414394e79c1a3c9ffafad862", log_level='error', server='demo', log_file='log.txt')

# return connection
if con.is_connected():
    print("Connect Successful!")

Connect Successful!


In [4]:
get_data = con.get_candles("EUR/USD", period='m5', number=50)[['bidopen','bidclose','bidhigh','bidlow']]
get_data.rename(columns = {'bidopen':'Open', 'bidhigh':'High', 'bidlow': 'Low', 'bidclose': 'Close'}, inplace = True)

In [30]:
res_ta = ta.trend.MACD(get_data['Close'], window_fast=12, window_slow=26 ,window_sign=9) # signal line is base and macd line will crossover
macd_line, signal_line = res_ta.macd()[-1:].values[0], res_ta.macd_signal()[-1:].values[0]

In [31]:
if macd_line > signal_line:
    signal, arrow_type = "BUY", "arrow-up"
elif macd_line < signal_line:
    signal, arrow_type = "SELL", "arrow-down"
else:
    signal, arrow_type = "NEUTRAL", "non-arrow"



('BUY', 'arrow-up')

In [35]:
res_ta = ta.momentum.RSIIndicator(get_data['Close'][-14:], window=14)

In [36]:
res_ta.rsi()

date
2022-05-03 10:40:00          NaN
2022-05-03 10:45:00          NaN
2022-05-03 10:50:00          NaN
2022-05-03 10:55:00          NaN
2022-05-03 11:00:00          NaN
2022-05-03 11:05:00          NaN
2022-05-03 11:10:00          NaN
2022-05-03 11:15:00          NaN
2022-05-03 11:20:00          NaN
2022-05-03 11:25:00          NaN
2022-05-03 11:30:00          NaN
2022-05-03 11:35:00          NaN
2022-05-03 11:40:00          NaN
2022-05-03 11:45:00    65.117649
Name: rsi, dtype: float64

In [6]:
indicators_signal = {
    "1": [
        {
            "timeframe": "10Min",
            "value": 1.05446,
            "signal": "SELL",
            "datetime": 1651592563000,
            "type": "trend"
        },
        {
            "timeframe": "1H",
            "value": 1.05213,
            "signal": "BUY",
            "datetime": 1651592563000,
            "type": "trend"
        }
    ],
    "2": [
        {
            "timeframe": "10Min",
            "arrowtype": "arrow-down",
            "value": 0.00009,
            "signal": "SELL",
            "datetime": 1651592563000,
            "type": "trend"
        },
        {
            "timeframe": "1H",
            "arrowtype": "arrow-up",
            "value": 0.00058,
            "signal": "BUY",
            "datetime": 1651592563000,
            "type": "trend"
        }
    ],
    "3": [
        {
            "timeframe": "10Min",
            "value": 32.5403,
            "signal": "NEUTRAL",
            "datetime": 1651592563000,
            "type": "momentum"
        },
        {
            "timeframe": "1H",
            "value": 54.35956,
            "signal": "NEUTRAL",
            "datetime": 1651592564000,
            "type": "momentum"
        }
    ]
}

In [7]:
return_summary = {}
return_summary['timeframe'] = {}
return_summary['all'] = {}
return_summary['all']['SELL'], return_summary['all']['BUY'] = 0, 0
for indicator in indicators_signal.keys():

    for tf in indicators_signal[indicator]:

        if tf['timeframe'] not in return_summary['timeframe']:
            return_summary['timeframe'][tf['timeframe']] = {}
            # initial trend 
            return_summary['trend'] = {}
            return_summary['timeframe'][tf['timeframe']]['trend'] = {}
            return_summary['timeframe'][tf['timeframe']]['trend']['BUY'], return_summary['trend']['BUY'] = 0, 0
            return_summary['timeframe'][tf['timeframe']]['trend']['SELL'], return_summary['trend']['SELL'] = 0, 0
            return_summary['timeframe'][tf['timeframe']]['trend']['NEUTRAL'], return_summary['trend']['NEUTRAL'] = 0, 0
            # initial momentum
            return_summary['momentum'] = {}
            return_summary['timeframe'][tf['timeframe']]['momentum'] = {}
            return_summary['timeframe'][tf['timeframe']]['momentum']['OVERBOUGHT'], return_summary['momentum']['OVERBOUGHT'] = 0, 0
            return_summary['timeframe'][tf['timeframe']]['momentum']['OVERSOLD'], return_summary['momentum']['OVERSOLD'] = 0, 0
            return_summary['timeframe'][tf['timeframe']]['momentum']['NEUTRAL'], return_summary['momentum']['NEUTRAL'] = 0, 0

        if tf['type'] == 'trend':
            return_summary['timeframe'][tf['timeframe']]['trend'][tf['signal']] += 1
            return_summary['trend'][tf['signal']] += 1
        elif tf['type'] == 'momentum':
            return_summary['timeframe'][tf['timeframe']]['momentum'][tf['signal']] += 1
            return_summary['momentum'][tf['signal']] += 1

        if tf['signal'] in ['BUY', 'OVERSOLD']: return_summary['all']['BUY'] += 1
        elif tf['signal'] in ['SELL', 'OVERBOUGHT']: return_summary['all']['SELL'] += 1
        
return_summary['summary'] = {}
if (return_summary['all']['BUY'] + return_summary['all']['SELL']) != 0:
    return_summary['summary']['BUY'] = return_summary['all']['BUY'] * 100 / (return_summary['all']['BUY'] + return_summary['all']['SELL'])
    return_summary['summary']['SELL'] = return_summary['all']['SELL'] * 100 / (return_summary['all']['BUY'] + return_summary['all']['SELL'])
else :
    return_summary['summary']['BUY'] = 0
    return_summary['summary']['SELL'] = 0

In [8]:
return_summary

{'timeframe': {'10Min': {'trend': {'BUY': 0, 'SELL': 2, 'NEUTRAL': 0},
   'momentum': {'OVERBOUGHT': 0, 'OVERSOLD': 0, 'NEUTRAL': 1}},
  '1H': {'trend': {'BUY': 2, 'SELL': 0, 'NEUTRAL': 0},
   'momentum': {'OVERBOUGHT': 0, 'OVERSOLD': 0, 'NEUTRAL': 1}}},
 'all': {'SELL': 2, 'BUY': 2},
 'trend': {'BUY': 2, 'SELL': 1, 'NEUTRAL': 0},
 'momentum': {'OVERBOUGHT': 0, 'OVERSOLD': 0, 'NEUTRAL': 2},
 'summary': {'BUY': 50.0, 'SELL': 50.0}}